In [129]:
#Imports
import pandas as pd
from scipy.stats import pearsonr

In [130]:
traffic_volume_df = pd.read_csv('../../data/raw/traffic-volume-intersections/raw-data-2020-2029.csv')
neighborhood_df = pd.read_excel('../../data/raw/neighbourhood-profiles-2021-158-model.xlsx')
intersectionNeighborhoodJoin_df = pd.read_csv('../../data/processed/proposal_exploration/intersectionsAndNeighborhoods.csv')

#Stripe white spaces in column name
neighborhood_df.columns = neighborhood_df.columns.str.strip()



In [131]:
neighborhood_df

,Neighbourhood Name,West Humber-Clairville,Mount Olive-Silverstone-Jamestown,Thistletown-Beaumond Heights,Rexdale-Kipling,Elms-Old Rexdale,Kingsview Village-The Westway,Willowridge-Martingrove-Richview,Humber Heights-Westmount,Edenbridge-Humber Valley,...,Harbourfront-CityPlace,St Lawrence-East Bayfront-The Islands,Church-Wellesley,Downtown Yonge East,Bay-Cloverhill,Yonge-Bay Corridor,Junction-Wallace Emerson,Dovercourt Village,North Toronto,South Eglinton-Davisville
0,Neighbourhood Number,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,174
1,TSNS 2020 Designation,Not an NIA or Emerging Neighbourhood,Neighbourhood Improvement Area,Neighbourhood Improvement Area,Not an NIA or Emerging Neighbourhood,Neighbourhood Improvement Area,Neighbourhood Improvement Area,Not an NIA or Emerging Neighbourhood,Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,...,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood,Not an NIA or Emerging Neighbourhood
2,Total - Age groups of the population - 25% sam...,33300,31345,9850,10375,9355,22005,22445,10005,15190,...,28135,31285,22320,17700,16670,12645,23180,12380,15885,22735
3,0 to 14 years,4295,5690,1495,1575,1610,3915,3500,1370,2070,...,2065,2285,895,1055,745,970,3075,1365,1315,2190
4,0 to 4 years,1460,1650,505,505,440,1245,1065,395,520,...,1030,1045,495,480,370,500,1135,445,535,910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2598,Total - Eligibility and instruction in the min...,3875,5540,1325,1370,1520,3460,3140,1275,2100,...,1185,1580,540,720,470,555,2375,1190,1050,1580
2599,Children eligible for instruction in the min...,335,395,120,90,70,240,240,70,225,...,145,200,60,70,80,30,305,130,95,155
2600,Eligible children who have been instructed...,255,245,75,75,60,150,185,45,130,...,70,165,35,55,75,10,190,95,65,105
2601,Eligible children who have not been instru...,75,145,45,25,10,95,55,25,95,...,75,35,25,15,0,20,115,35,30,45


### Select features that are of interest in the neighborhood DF.
#### Features:
1. Number of people 15 to 64 years
2. Number of people aged 65 and older
3. Median total income in 2020  among recipients ($)
4. Number of people with total income...:
    Under $10,000 (including loss)
    $10,000 to $19,999
    $20,000 to $29,999
    $30,000 to $39,999
    $40,000 to $49,999
    $50,000 to $59,999
    $60,000 to $69,999
    $70,000 to $79,999
    $80,000 to $89,999
    $90,000 to $99,999
    $100,000 and over
    $100,000 to $149,999
    $150,000 and over
5. Number of people "In the labour force"
6. Number of people with the main mode of commuting as..: Total - Main mode of commuting for the employed labour force aged 15 years and over with a usual place of work or no fixed workplace address - 25% sample data
    Car, truck or van
    Car, truck or van - as a driver
    Car, truck or van - as a passenger
    Public transit
    Walked
    Bicycle
7. Total amount of people who leave for work during the time...: Total - Time leaving for work for the employed labour force aged 15 years and over with a usual place of work or no fixed workplace address - 25% sample data
    Between 5 a.m. and 5:59 a.m.
    Between 6 a.m. and 6:59 a.m.
    Between 7 a.m. and 7:59 a.m.
    Between 8 a.m. and 8:59 a.m.
    Between 9 a.m. and 11:59 a.m.
    Between 12 p.m. and 4:59 a.m.

#### Notes:
For now we are going to not include the 7th set of features (Total amount of people who leave for work during the time), as it relates to a time series, which can be explored at a later date.

In [132]:
#Keep certain rows before transposing. These rows are the features mentioned.
#indices_to_keep = [7, 18, 62, 1964, 2575, 2578, 2579, 2580, 2581, 2589, 2590,2591,2592,2593,2594]
indices_to_keep = [0, 62, 1964]

neighborhood_df_mod = neighborhood_df.loc[indices_to_keep]

#Switch rows with columns.
neighborhood_df_mod = neighborhood_df_mod.transpose()

# Set first row as column header in DF.
neighborhood_df_mod.columns = neighborhood_df_mod.iloc[0]

# Remove the first row since it's now the column header
neighborhood_df = neighborhood_df_mod.iloc[1:]

neighborhood_df

Neighbourhood Name,Neighbourhood Number,Median total income in 2020 among recipients ($),In the labour force
West Humber-Clairville,1,33600,18405
Mount Olive-Silverstone-Jamestown,2,29600,14360
Thistletown-Beaumond Heights,3,32800,4990
Rexdale-Kipling,4,33600,5305
Elms-Old Rexdale,5,34400,4425
...,...,...,...
Yonge-Bay Corridor,170,44000,8620
Junction-Wallace Emerson,171,41200,14040
Dovercourt Village,172,38000,7460
North Toronto,173,46000,10870


### Processing traffic volume DF.
We are going to remove columns that don't matter to us


In [133]:
traffic_volume_df =  pd.read_csv('../../data/raw/traffic-volume-intersections/raw-data-2020-2029.csv')

columns = [0,1,2,4,5,6,7,8,9,10,11]


for i, col in enumerate(traffic_volume_df.columns):
    print(f"Column {i}: {col}")

traffic_volume_df = traffic_volume_df.drop(traffic_volume_df.columns[columns], axis=1)



Column 0: _id
Column 1: count_id
Column 2: count_date
Column 3: location_id
Column 4: location
Column 5: lng
Column 6: lat
Column 7: centreline_type
Column 8: centreline_id
Column 9: px
Column 10: time_start
Column 11: time_end
Column 12: sb_cars_r
Column 13: sb_cars_t
Column 14: sb_cars_l
Column 15: nb_cars_r
Column 16: nb_cars_t
Column 17: nb_cars_l
Column 18: wb_cars_r
Column 19: wb_cars_t
Column 20: wb_cars_l
Column 21: eb_cars_r
Column 22: eb_cars_t
Column 23: eb_cars_l
Column 24: sb_truck_r
Column 25: sb_truck_t
Column 26: sb_truck_l
Column 27: nb_truck_r
Column 28: nb_truck_t
Column 29: nb_truck_l
Column 30: wb_truck_r
Column 31: wb_truck_t
Column 32: wb_truck_l
Column 33: eb_truck_r
Column 34: eb_truck_t
Column 35: eb_truck_l
Column 36: sb_bus_r
Column 37: sb_bus_t
Column 38: sb_bus_l
Column 39: nb_bus_r
Column 40: nb_bus_t
Column 41: nb_bus_l
Column 42: wb_bus_r
Column 43: wb_bus_t
Column 44: wb_bus_l
Column 45: eb_bus_r
Column 46: eb_bus_t
Column 47: eb_bus_l
Column 48: nx_pe

### Join the intersection data with nieghborhood data.
#### This will be split into two stages:
##### 1. Simplifying traffic flow volume
This will entail summing traffic colume counts per all time intervals per intersection, and summing all modes and directions of traffic flow.
##### 2. Joining neighborhood data
This will be done by joining the neighborhood data onto the intersection data.  

### Simplying traffic flow volume

In [134]:
# Sum values across rows grouped by 'location_id'. Reset index keeps column labels
grouped_time_intervals_trafficFlow = traffic_volume_df.groupby('location_id').sum().reset_index()

# Sum values across columns.
trafficFlow_locationID = grouped_time_intervals_trafficFlow.iloc[:,0].reset_index()

# Summation of all modes of traffic and directional volume
grouped_time_intervals_trafficFlow
summation_trafficFlow = pd.DataFrame(grouped_time_intervals_trafficFlow.iloc[:,1:].sum(axis=1), columns = ['total_volume'])

# Adding location ID to volume record
trafficFlow = pd.concat([trafficFlow_locationID, summation_trafficFlow], axis=1)

trafficFlow = trafficFlow.drop(trafficFlow.columns[0], axis=1)

trafficFlow


,location_id,total_volume
0,1146,24142
1,3468,14107
2,3925,9958
3,3926,40338
4,3934,12286
...,...,...
2686,50112,2928
2687,50113,1218
2688,50114,5277
2689,50116,335


#### Joining traffic volume with neighborhood features we selected earlier.

In [135]:
junction_df = pd.read_csv('../../data/processed/proposal_exploration/intersectionsAndNeighborhoods.csv')

#Select only two columns we need as join keys
junction_df = junction_df.iloc[:,[3,11]]

#junction_df

traffic_volume_intersection_df = pd.merge(junction_df, trafficFlow, how="inner", on=['location_id', 'location_id'])

traffic_volume_intersection_df.sort_values('neighbourhood_id')

,location_id,neighbourhood_id,total_volume
1651,30709,1.0,9321
1388,13554,1.0,4735
2260,42443,1.0,2688
654,5123,1.0,17253
1505,25111,1.0,3944
...,...,...,...
1705,31901,NaN,8783
1725,32666,NaN,25750
2230,42411,NaN,18645
2524,43415,NaN,22699


Further processing to obtain the final DF to run the regression on.

In [136]:
# Drop all NAN values for niegborhood ID
traffic_volume_intersection_df = traffic_volume_intersection_df.dropna(subset=['neighbourhood_id'])

# Convert all values to int
traffic_volume_intersection_df['neighbourhood_id'] = traffic_volume_intersection_df['neighbourhood_id'].astype(int)

# Rename column label to exclude spaces. Otherwise, merge will not work.
neighborhood_df = neighborhood_df.rename(columns={'Neighbourhood Number': 'neighbourhood_id'})

print(traffic_volume_intersection_df.columns)
print(neighborhood_df.columns)

# Merge 
traffic_volume_intersection_df = pd.merge(traffic_volume_intersection_df, neighborhood_df, how="outer", on=['neighbourhood_id', 'neighbourhood_id'])

# Rename column labels to removes spaces
traffic_volume_intersection_df.columns = traffic_volume_intersection_df.columns.str.replace(' ', '_')
print(traffic_volume_intersection_df.columns)

# Mapping for column renaming
column_mapping = {
    '____Median_total_income_in_2020__among_recipients_($)': 'total_median_income',
    '__In_the_labour_force': 'labor_force_amount',
    'total_volume':'total_traffic_volume'
}

df = traffic_volume_intersection_df.rename(columns=column_mapping)

df.to_csv('../../data/processed/proposal_exploration/regression_intersection_neighborhood_features.csv')

Index(['location_id', 'neighbourhood_id', 'total_volume'], dtype='object')
Index(['neighbourhood_id',
       '    Median total income in 2020  among recipients ($)',
       '  In the labour force'],
      dtype='object', name='Neighbourhood Name')
Index(['location_id', 'neighbourhood_id', 'total_volume',
       '____Median_total_income_in_2020__among_recipients_($)',
       '__In_the_labour_force'],
      dtype='object')


### Multivariate Multiple Regression 
We will use this DF to test that all these features are dependant variables based on the flow of traffic.
We want to assess the strength and direction of the linear relationship between a few continuous variables, but we are primarily interested in exploring associations between variables rather than predicting outcomes.